# Crise du sida Berlin
Ce notebook Jupiter contient un **[tableau](#Tableau)** de tous les lieux visités pendant l'atelier topographique sur la crise du sida à Berlin dans les années 80 (*appel à participation [DE](https://www.sciencespo.fr/histoire/fr/node/89436.html) | [FR](https://cmb.hu-berlin.de/fr/agenda/evenement/topografischer-geschichtsworkshop-fuer-junge-forscherinnen)*), et une **[carte dynamique](#Carte)** générée à partir de ce tableau. 

L'atelier s'est tenu au Marc Bloch Zentrum du 13 au 18 mai 2024 et a entre autres donné lieu à un [interview de l'une des organisatrices](https://www.sciencespo.fr/fr/actualites/la-crise-du-sida-a-berlin-parcourir-une-ville-pour-comprendre-son-histoire/), la création d'une [librairie Zotero](https://www.zotero.org/groups/5537257/aidskrisebln2024/library) et d'un [répertoire de travail commun](https://sharedocs.huma-num.fr/wl/?id=Su3ebB9UxlLW3jueknVkCwTB4B0ckciH), afin d'archiver ce travail sur l'archive et de le poursuivre plus tard. 

Le présent code source est sous licence [CC0 1.0 Deed](https://creativecommons.org/publicdomain/zero/1.0/deed.fr). Les données sont en cours de dépôt sur [RechercheDataGouv](https://entrepot.recherche.data.gouv.fr/dataverse/root/?q=)

## Tableau 

Ce tableau est éditable en .xlsx sur le [répertoire ](https://sharedocs.huma-num.fr/wl/?id=on6pmqexVsfysrKr8E8QwjktVP7gaABW&fmode=edit). 

In [12]:
import pandas as pd
import folium

df_raw = pd.read_csv("synth-lieux.csv", encoding='utf-8', sep=';')
df = df_raw[df_raw['Visited'] == "oui"]

# Tableau simplifié pour affichage dans le notebook
df_simple = df[["Place", "AddrPlain", "Type", "Community", "East or west"]]
df_simple = df_simple.style.set_properties(**{'text-align': 'left'}).set_table_styles(
    [{'selector': 'th', 'props': [('text-align', 'left')]}]
)
df_simple


,Place,AddrPlain,Type,Community,East or west
0,Centre Marc Bloch,"Friedrichstraße 177-191, 10117 Berlin, Allemagne",Research Center,"Researchers, queer activists",East
1,Chez Michel,"Adalbertstraße 83, 10997 Berlin, Allemagne",Restaurant,French,West
2,Yorkstrasse,"Großgörschenstraße 30, 10827 Berlin, Allemagne",Train station,"Lesbians, gays, queers",West
3,Les Migras,"Kulmer Str. 20A, 10783 Berlin, Allemagne",Community center,"Lesbians, migrants",West
4,Lesbenberatung,"Kulmer Str. 20A, 10783 Berlin, Allemagne",Community center,Lesbians,West
5,Siegesaüle,"Kulmer Str. 20A, 10783 Berlin, Allemagne",Magazine,Gays,West
6,Begine - Frauenkneipe und Kultur,"Potsdamer Str. 139, 10783 Berlin, Allemagne",Squat,Lesbians,West
7,Blocksberg,"Yorckstraße 47-48, 10783 Berlin, Allemagne",Bar,Lesbians,West
8,Kaiserhof,"Bülowstraße 36, 10783 Berlin, Allemagne",Party,"Lebians, trans persons, gays",West
9,Isherwood,"Nollendorfstraße 17, 10777 Berlin, Allemagne",Individual (writer),Gays,West


## Carte

In [13]:
coords= df["Adresse"].tolist()
places = df["Place"].tolist()
typologies = df["Type"].tolist()
communities = df["Community"].tolist()
descriptions = df["Description"].tolist()
images = df["Pics"].tolist()

styling="""<style="color: rgba(251, 238, 230, 1);">"""

# coordonnées 
cleaned_coords = [(lat, lon) for coord in coords if isinstance(coord, str) and ',' in coord for lat, lon in [coord.split(',')]]

coords_with_places = [(coord[0], coord[1], place, typology, community, description, image) for coord, place, typology, community, description, image in zip(cleaned_coords, places, typologies, communities, descriptions, images)]

# Create a map centered at the first coordinate
map_center = (float(coords_with_places[0][0]), float(coords_with_places[0][1]))
my_map = folium.Map(location=map_center, zoom_start=13)

# CSS pour le style des cartels
styling_h1 = """<span style="color: rgb(251,153,142); font-weight: 900; font-size: 150%;">"""
styling_h2 = """<span style="color: rgb(253,193,186); font-weight: 600; font-size: 120%;">"""
styling_desc = """<p style="color: rgb(85,85,85); font-weight: 300; font-size: 120%;">"""


# Add markers for each coordinate
for loc in coords_with_places:
    lat, lon, place, typologies, communities, descriptions, images = loc
    folium.Marker(
        location=(float(lat), float(lon)),
        popup=f"""{styling_h1}{place}</span><br>
        {styling_h2}Type</span> {typologies}<br>
        {styling_h2}Communities</span> {communities}<br><br>
        {styling_h2} Description </span><br> {styling_desc}{descriptions}</p><br>
        {styling_h2}Street view</span> {images}
        """
    ).add_to(my_map)

# Save the map to an HTML file
my_map
